# 🥋 Lekcja 6: Einops (Czytelne Tensory)

Pisanie `x.view(b, h*w, c)` jest niebezpieczne. Jeśli pomylisz kolejność wymiarów, kod zadziała, ale model będzie uczył się bzdur (Silent Bug).

**Einops** to biblioteka, która wprowadza **programowanie deklaratywne** dla tensorów.
Mówisz *"co chcesz uzyskać"*, a nie *"jak przesunąć bajty"*.

Trzy główne funkcje:
1.  **`rearrange`**: Zastępuje `view`, `reshape`, `permute`, `transpose`, `squeeze`, `unsqueeze`. Wszystko w jednym.
2.  **`reduce`**: Zastępuje `mean`, `sum`, `max` z obsługą wymiarów.
3.  **`repeat`**: Zastępuje `repeat`, `expand`.

Zainstalujmy ją i naprawmy czytelność kodu.

In [ ]:
# Instalacja (jeśli nie masz w środowisku)
!uv pip install einops

import torch
from einops import rearrange, reduce, repeat

# 1. DANE (Batch obrazków RGB)
# [Batch=16, Channels=3, Height=32, Width=32]
images = torch.randn(16, 3, 32, 32)

print(f"Dane wejściowe: {images.shape} (B, C, H, W)")

## 1. `rearrange`: Szwajcarski Scyzoryk

Zapomnij o `permute` i `view`.

**Scenariusz 1: Zamiana kanałów (HWC <-> CHW)**
Standard w wizji komputerowej. OpenCV lubi HWC, PyTorch lubi CHW.

In [4]:
# Klasycznie w PyTorch (Mało czytelne)
# Musisz pamiętać, że dim 1 to kanały, 2 to wysokość...
y_torch = images.permute(0, 2, 3, 1)

# Einops (Czytelne!)
# 'b c h w -> b h w c'
y_einops = rearrange(images, 'b c h w -> b h w c')

print(f"PyTorch: {y_torch.shape}")
print(f"Einops:  {y_einops.shape}")
assert torch.allclose(y_torch, y_einops)

PyTorch: torch.Size([16, 32, 32, 3])
Einops:  torch.Size([16, 32, 32, 3])


## Scenariusz 2: Patching (Vision Transformer)

To najtrudniejsza operacja w ViT (Notatnik 70).
Musimy pociąć obrazek na kwadraty (patche) i spłaszczyć je.

Obrazek: `(b, c, h, w)`
Cel: `(b, liczba_patchy, rozmiar_patcha)`

W czystym PyTorch to koszmar (`unfold`, `view`, `permute`).
W Einops to jedna linijka.

In [5]:
# Chcemy pociąć obrazek 32x32 na patche 8x8.
# Ile będzie patchy? (32/8) * (32/8) = 4 * 4 = 16 patchy.
# Rozmiar jednego patcha (spłaszczonego): 3 kanały * 8 * 8 = 192.

# h -> (h1 h2), gdzie h2=8 (wysokość patcha)
# w -> (w1 w2), gdzie w2=8 (szerokość patcha)
patch_size = 8

patches = rearrange(
    images, 
    'b c (h h2) (w w2) -> b (h w) (c h2 w2)', 
    h2=patch_size, 
    w2=patch_size
)

print(f"Patche: {patches.shape}")
# Oczekujemy: [16 batch, 16 patchy, 192 wymiar]

Patche: torch.Size([16, 16, 192])


## 2. `reduce`: Agregacja

Zastępuje `torch.mean` czy `torch.sum`, ale jest bezpieczniejsze, bo nazywasz wymiary.

**Scenariusz:** Global Average Pooling.
Mamy `(Batch, C, H, W)`. Chcemy średnią po pikselach, żeby dostać `(Batch, C)`.

In [6]:
# Klasycznie
# mean(dim=(2, 3)) - trzeba liczyć indeksy
gap_torch = images.mean(dim=(2, 3))

# Einops
# "Zredukuj wysokość i szerokość do pojedynczego punktu, używając średniej"
gap_einops = reduce(images, 'b c h w -> b c', 'mean')

print(f"GAP shape: {gap_einops.shape}")
assert torch.allclose(gap_torch, gap_einops)

# BONUS: Max Pooling po kanałach (jaki jest najjaśniejszy piksel w każdym punkcie?)
max_val_per_pixel = reduce(images, 'b c h w -> b h w', 'max')
print(f"Max Channel shape: {max_val_per_pixel.shape}")

GAP shape: torch.Size([16, 3])
Max Channel shape: torch.Size([16, 32, 32])


## 3. `repeat`: Rozgłaszanie

Kiedy chcesz powielić dane (np. dodać ten sam `bias` do każdego piksela).

Mamy wektor `(Batch)`. Chcemy go powielić do `(Batch, H, W)`.

In [7]:
# Wektor klas dla batcha
labels = torch.arange(16) # [0, 1, 2... 15]

# Chcemy stworzyć "maskę", gdzie każdy piksel obrazka ma wartość klasy
# (b) -> (b h w)

mask = repeat(labels, 'b -> b h w', h=32, w=32)

print(f"Labels: {labels.shape}")
print(f"Mask:   {mask.shape}")

print(f"Próbka 0, piksel 0,0: {mask[0,0,0]}") # Powinno być 0
print(f"Próbka 5, piksel 10,10: {mask[5,10,10]}") # Powinno być 5

Labels: torch.Size([16])
Mask:   torch.Size([16, 32, 32])
Próbka 0, piksel 0,0: 0
Próbka 5, piksel 10,10: 5


## 🥋 Black Belt Summary

**Dlaczego Einops to "Black Belt"?**
1.  **Czytelność:** Kod dokumentuje się sam. Widzisz `'b c h w -> b h w c'` i wiesz, co się dzieje.
2.  **Bezpieczeństwo:** Jeśli wymiary się nie zgadzają (np. obrazek nie dzieli się równo na patche), Einops rzuci czytelnym błędem od razu. PyTorch `view()` po prostu przemieli dane i wypluje śmieci.
3.  **Uniwersalność:** Działa tak samo w PyTorch, TensorFlow, JAX i NumPy.

**Zadanie:** W następnym projekcie, gdy będziesz chciał użyć `.view()`, zatrzymaj się i użyj `rearrange()`.